Importing Libraries

In [3]:
import random

In [4]:
class TicTacToe:
    def __init__(self):
        self.board = [" " for _ in range(9)]
        self.current_winner = None

    def reset(self):
        self.board = [" " for _ in range(9)]
        self.current_winner = None        

    def print_board(self):
        print("   0   1   2")
        for i in range(3):
            row = self.board[i*3:(i+1)*3]
            # print(row)
            print(f"{i} | {row[0]} | {row[1]} | {row[2]} |")
            if i < 2:
                print("  ---+---+---")

    def move(self, square, letter):
        if self.board[square] == " ":
            self.board[square] = letter
            if self.check_winner(square, letter):
                self.current_winner = letter
            return True
        return False
    
    def check_winner(self, square, letter):
        row_ind = square // 3
        row = self.board[row_ind * 3:(row_ind + 1) * 3]
        if all([s == letter for s in row]):
            return True     

        col_ind = square % 3
        col = [self.board[col_ind + i * 3] for i in range(3)]
        if all([s == letter for s in col]):
            return True

        if square % 2 == 0:
            diag1 = [self.board[i] for i in [0, 4, 8]]
            if all([s == letter for s in diag1]):
                return True

            diag2 = [self.board[i] for i in [2, 4, 6]]
            if all([s == letter for s in diag2]):
                return True

        return False           


    def empty_squares(self):
        return " " in self.board

    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == " "]

    def is_full(self):
        return not self.empty_squares()

    def check_draw(self):
        return self.is_full() and self.current_winner is None


In [5]:
class RandomComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        return random.choice(game.available_moves())
